# 微信恋爱报告

In [77]:
import pandas as pd 
import os 


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
df = pd.read_csv("./妠妠老婆🍪.csv")
df = df[["StrContent", "StrTime", "Remark"]]
df = df.astype(str)
os.makedirs("./mzz_data/", exist_ok=True)

# 将这里替换为你们的微信昵称
remark_1 = "小老正"
remark_2 = "妠妠老婆🍪"

# 这里替换为你们的姓名简写
name_1 = "mzz"
name_2 = "syn"

## 发送表情包的次数

In [ ]:
df["is_emoji"] = df["StrContent"].apply(lambda x: "emoji" in x)
# 总的表情包发送次数
print("总的表情包发送次数：" + str(df.loc[df["is_emoji"]==True, "StrContent"].count()))
# 小老正的表情包发送次数
print(f"{remark_1}的表情包发送次数：" + str(df.loc[(df["is_emoji"]==True) & (df["Remark"]==remark_1), "StrContent"].count()))
# 妠妠老婆🍪的表情包发送次数
print(f"{remark_2}的表情包发送次数：" + str(df.loc[(df["is_emoji"]==True) & (df["Remark"]==remark_2), "StrContent"].count()))

In [ ]:
import matplotlib.pyplot as plt 


# 统计表情包发送次数
total_emoji = df.loc[df["is_emoji"] == True, "StrContent"].count()
mzz_emoji = df.loc[(df["is_emoji"] == True) & (df["Remark"] == remark_1), "StrContent"].count()
syn_emoji = df.loc[(df["is_emoji"] == True) & (df["Remark"] == remark_2), "StrContent"].count()
# 数据准备
labels = [name_1, name_2]  # 使用简写
sizes = [mzz_emoji, syn_emoji]  # 各部分的大小
colors = ["#FFB6C1", "#FFF5E1"]  # 粉色和奶油色
explode = (0, 0.1)  # 突出显示前两部分

# 启用手绘风格
plt.xkcd()

# 绘制饼图
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", startangle=90, shadow=True)

# 设置英文标题
plt.title("Pie of Emoji", fontsize=16, fontweight="bold")

# 保存
plt.savefig("./mzz_data/表情包发送次数饼图.png", dpi=300)
# 显示图表
plt.show()

## 最常发送的表情包

In [ ]:
import os
import requests
import xml.etree.ElementTree as ET

# 下载表情包图片的函数
def download_emoji_image(emoji_xml, dir_path, rank, count):
    # 解析XML
    root = ET.fromstring(emoji_xml)

    # 找到emoji标签
    emoji_element = root.find('.//emoji')
    if emoji_element is None:
        raise ValueError("未找到emoji标签")

    # 获取cdnurl
    cdnurl = emoji_element.get('cdnurl')
    if not cdnurl:
        raise ValueError("cdnurl未找到")

    # 下载图片
    response = requests.get(cdnurl)
    if response.status_code != 200:
        raise ValueError(f"下载失败，状态码: {response.status_code}")

    # 保存图片，命名为“排名_发送次数.jpg”
    filename = os.path.join(dir_path, f"{rank}_{count}.jpg")
    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f"图片已下载并保存为 {filename}")

# 创建目录
dir_path = f"./mzz_data/emoji/{remark_2}最常发的图片"
os.makedirs(dir_path, exist_ok=True)

# 获取最常发的表情包XML列表及其发送次数
emoji_stats = df.loc[(df["is_emoji"] == True) & (df["Remark"] == remark_2), "StrContent"].value_counts().head(10)

# 遍历每个表情包XML，下载并命名
for rank, (emoji_xml, count) in enumerate(emoji_stats.items()):
    download_emoji_image(emoji_xml, dir_path, rank, count)

# 创建目录
dir_path = f"./mzz_data/emoji/{remark_1}最常发的图片"
os.makedirs(dir_path, exist_ok=True)

# 获取最常发的表情包XML列表及其发送次数
emoji_stats = df.loc[(df["is_emoji"] == True) & (df["Remark"] == remark_1), "StrContent"].value_counts().head(10)

# 遍历每个表情包XML，下载并命名
for rank, (emoji_xml, count) in enumerate(emoji_stats.items()):
    download_emoji_image(emoji_xml, dir_path, rank, count)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings


# 启用 xkcd 模式
plt.xkcd()

# 设置字体为黑体，解决中文显示问题
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示为方块的问题


# 将 StrTime 转换为 datetime 类型
df["StrTime"] = pd.to_datetime(df["StrTime"])

# 提取小时信息
df["Hour"] = df["StrTime"].dt.hour

import re

# 定义函数，统计汉字长度
def count_chinese_characters(text):
    # 匹配所有中文字符（包括简体和繁体）
    chinese_characters = re.findall(r'[\u4e00-\u9fff]', text)
    return len(chinese_characters)

# 应用到 DataFrame 中
df["WordCount"] = df["StrContent"].apply(count_chinese_characters)

# 按小时统计聊天字数
hourly_word_count = df.groupby("Hour")["WordCount"].sum()

# 可视化
plt.figure(figsize=(10, 6))

# 使用 plt.bar 绘制柱状图
plt.bar(hourly_word_count.index, hourly_word_count.values, color="mediumseagreen", edgecolor="black", linewidth=2, alpha=0.7)

# 设置标题和标签
plt.title("Word Count Per Hour", fontsize=16)
plt.xlabel("Hour", fontsize=12)
plt.ylabel("Word Count", fontsize=12)

# 设置 x 轴刻度
plt.xticks(range(24), [f"{i}:00" for i in range(24)], rotation=45)

# 调整布局
plt.tight_layout()

# 保存
plt.savefig("./mzz_data/hourly_word_count.png")
# 显示图表
plt.show()

# 聊天日期Github热力图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar

# 将 StrTime 转换为 datetime 类型
df["StrTime"] = pd.to_datetime(df["StrTime"])

# 提取日期
df["Date"] = df["StrTime"].dt.date

# 确保 Date 列为 datetime 类型
df['Date'] = pd.to_datetime(df['Date'])

# 过滤2024年的数据
df_2024 = df[df['Date'].dt.year == 2024]

# 按日期统计聊天字数
daily_word_count = df_2024.groupby('Date')['WordCount'].sum().reset_index()

# 使用箱线图去掉异常值
Q1 = daily_word_count['WordCount'].quantile(0.25)
Q3 = daily_word_count['WordCount'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

# 异常值填补为边界
daily_word_count.loc[daily_word_count['WordCount'] > upper_bound, 'WordCount'] = upper_bound

# 为热力图创建矩阵
daily_word_count['Weekday'] = daily_word_count['Date'].dt.weekday  # 星期几
daily_word_count['Week'] = daily_word_count['Date'].dt.strftime('%U')  # 第几周

# 创建一个全年的空矩阵 (52周 x 7天)
heatmap_data = pd.DataFrame(
    index=range(7), columns=range(1, 53), data=0
)

# 填充矩阵
for _, row in daily_word_count.iterrows():
    week = int(row['Week'])
    weekday = row['Weekday']
    heatmap_data.loc[weekday, week] = row['WordCount']


# 获取每个月的起始日期对应的周数
first_day_of_month = pd.date_range(start='2024-01-01', end='2024-12-31', freq='MS')
month_labels = {int(date.strftime('%U')): date.strftime('%b') for date in first_day_of_month}

# 修改绘图代码
plt.figure(figsize=(20, 8))
sns.heatmap(
    heatmap_data,
    cmap='Greens',
    linewidths=0.5,
    linecolor='white',
    cbar_kws={'label': 'Word Count'},
    square=True
)


# 替换横坐标的刻度标签
weeks = range(0, 53)
xtick_labels = [month_labels[week] if week in month_labels else '' for week in weeks]
plt.xticks(ticks=range(0, 53), labels=xtick_labels, rotation=0)
# 设置标题和其他样式
plt.title('Chat Activity Heatmap', fontsize=16)
plt.xlabel('Month')
plt.ylabel('Weekday')
plt.yticks(ticks=range(7), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], rotation=0)
plt.tight_layout()
plt.savefig("./mzz_data/heatmap.png")
plt.show()

## 聊天字数

In [ ]:
# 总聊天次数
print("总聊天次数：" + str(df["WordCount"].sum()))
# 小老正的聊天字数
print(f"{remark_1}的聊天字数：" + str(df.loc[df["Remark"]==remark_1, "WordCount"].sum()))
# 妠妠老婆🍪的聊天字数
print(f"{remark_2}的聊天字数：" + str(df.loc[df["Remark"]==remark_2, "WordCount"].sum()))

In [ ]:
import matplotlib.pyplot as plt 


# 统计表情包发送次数
total_word = df["WordCount"].sum()
mzz_word = df.loc[df["Remark"]==remark_1, "WordCount"].sum()
syn_word = df.loc[df["Remark"]==remark_2, "WordCount"].sum()
# 数据准备
labels = [name_1, name_2]  # 使用简写
sizes = [mzz_word, syn_word]  # 各部分的大小
colors = ["#FFB6C1", "#FFF5E1"]  # 粉色和奶油色
explode = (0, 0.1)  # 突出显示前两部分

# 启用手绘风格
plt.xkcd()

# 绘制饼图
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", startangle=90, shadow=True)

# 设置英文标题
plt.title("Pie of word", fontsize=16, fontweight="bold")

# 保存
plt.savefig("./mzz_data/字数次数饼图.png", dpi=300)
# 显示图表
plt.show()

## 聊天时间段

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 将 StrTime 转换为 datetime 类型
df["StrTime"] = pd.to_datetime(df["StrTime"])

# 提取日期
df["Date"] = df["StrTime"].dt.date

# 统计每日消息数量
daily_message_count = df.loc[df["is_emoji"]==False].groupby("Date").size()

# 看看聊天最频繁是哪天
print("聊天最频繁是哪天：" + str(daily_message_count.index[daily_message_count.argsort()[::-1][1]]))

# 看看这天说了什么
text = "\n".join(df.loc[df["Date"]==daily_message_count.index[daily_message_count.argsort()[::-1][1]]]["StrContent"].values)
# 正则表达式匹配中文字符和标点符号，同时保留换行符
pattern = re.compile(r'[^\u4e00-\u9fff\u3000-\u303f\uff00-\uffef\n:]')

# 替换掉非中文字符、标点符号和换行符
filtered_text = re.sub(pattern, '', text)

# 去除多余的空行（保留单行换行）
filtered_text = re.sub(r'\n+', '\n', filtered_text).strip()

# 这天说了多少字
print("这天说了多少字：" + str(len(filtered_text)))
# 这天说了什么
print("这天说了什么：\n" + filtered_text)

## 聊天字数随时间变化折线图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



# 将 StrTime 转换为 datetime 类型
df['StrTime'] = pd.to_datetime(df['StrTime'])

# 过滤出你和女朋友的聊天记录
syn_df = df[df['Remark'] == remark_2]  # 女朋友的聊天记录
mzz_df = df[df['Remark'] == remark_1]      # 你的聊天记录

# 按月份统计字数
syn_df['Month'] = syn_df['StrTime'].dt.to_period('M')
mzz_df['Month'] = mzz_df['StrTime'].dt.to_period('M')

syn_monthly_word_count = syn_df.groupby('Month')['WordCount'].sum().iloc[: -1]
mzz_monthly_word_count = mzz_df.groupby('Month')['WordCount'].sum().iloc[: -1]

# 找到最高点的月份和字数
syn_max_month = syn_monthly_word_count.index.astype(str)[syn_monthly_word_count.argmax()]
syn_max_word_count = syn_monthly_word_count.max()

mzz_max_month = mzz_monthly_word_count.index.astype(str)[mzz_monthly_word_count.argmax()]
mzz_max_word_count = mzz_monthly_word_count.max()

# 启用 xkcd 风格
plt.xkcd()

# 绘制折线图
fig, ax = plt.subplots(figsize=(12, 7))

# 绘制女朋友的折线图
ax.plot(syn_monthly_word_count.index.astype(str), syn_monthly_word_count.values, 
        marker='o', linestyle='-', color='lightcoral', linewidth=2, markersize=8, label=f'{name_2.upper()}\'s Word Count')

# 绘制你的折线图
ax.plot(mzz_monthly_word_count.index.astype(str), mzz_monthly_word_count.values, 
        marker='o', linestyle='-', color='mediumseagreen', linewidth=2, markersize=8, label=f'{name_1.upper()}\'s Word Count')

# 标记女朋友的最高点
ax.plot(syn_max_month, syn_max_word_count, marker='*', markersize=20, color='coral', 
        markeredgecolor='black', markeredgewidth=1, label=f'{name_2.upper()}\'s Highest Point')

# 标记你的最高点
ax.plot(mzz_max_month, mzz_max_word_count, marker='*', markersize=20, color='gold', 
        markeredgecolor='black', markeredgewidth=1, label=f'{name_1.upper()}\'s Highest Point')

# 设置标题和标签
ax.set_title('Word Count Over Time', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Word Count', fontsize=12)

# 旋转 x 轴标签
plt.xticks(rotation=45)

# 显示网格
ax.grid(True, linestyle='-', alpha=0.6)

# 显示图例
ax.legend()

# 自动调整布局
plt.tight_layout()

# 保存
plt.savefig('./mzz_data/combined_word_count_over_time.png', dpi=300)

# 显示图表
plt.show()

2024年5月，我们进入热恋期，聊天字数达到了最高点（一些照片）
2024年7月，老公放暑假啦，我们短暂的结束了异地时光，经常出去玩，聊天字数有所下降
之后，我们的聊天字数持续上升，说明我们感情越来越好！

## 对话情感分析

In [12]:
import json
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm 

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX",  # 替换为你的 API Key
    base_url="https://api.deepseek.com",
)


# 将 StrTime 转换为 datetime 类型
df['StrTime'] = pd.to_datetime(df['StrTime'])

# 按天分组，拼接对话内容
daily_conversations = df.groupby('Date').apply(lambda x: ' '.join(x['StrContent'])).reset_index(name='Conversation')

# 定义 system_prompt，引导模型输出 JSON 格式的情感分析结果
system_prompt = """
你是一个情感分析助手。用户会提供一段对话内容，请你分析这段对话的情感倾向，并输出 JSON 格式的结果。

输出格式要求：
{
    "sentiment": "甜蜜" | "吵架" | "平淡"
}

分析规则：
1. 如果对话中包含大量积极、温馨、甜蜜的内容（例如“爱你”、“想你”、“开心”等），则输出“甜蜜”。
2. 如果对话中包含大量负面、争吵、不满的内容（例如“生气”、“讨厌”、“烦”等），则输出“吵架”。
3. 如果对话内容主要是日常交流，没有明显的情感倾向，则输出“平淡”。

请根据对话内容，严格按照上述规则输出 JSON 格式的结果。
"""

# 定义重试机制
def analyze_sentiment_with_retry(conversation, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": conversation}
                ],
                response_format={"type": "json_object"}
            )
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(delay)
    return {"sentiment": "平淡"}  # 如果重试多次失败，默认返回“平淡”

# 分析每一天的情感倾向
sentiment_counts = {"甜蜜": 0, "吵架": 0, "平淡": 0}

for index, row in tqdm(daily_conversations.iterrows(), total=daily_conversations.shape[0]):
    date = row['Date']
    conversation = row['Conversation']
    
    # 调用 API 分析情感
    result = analyze_sentiment_with_retry(conversation)
    sentiment = result.get("sentiment", "平淡")  # 如果返回结果中没有 sentiment，默认设为“平淡”
    
    # 统计情感天数
    sentiment_counts[sentiment] += 1
    
    # 打印结果
    print(f"日期: {date}, 情感: {sentiment}")

# 打印最终统计结果
print("\n情感统计结果:")
for sentiment, count in sentiment_counts.items():
    print(f"{sentiment}: {count} 天")

In [ ]:
import matplotlib.pyplot as plt


# 标签和数据
labels = sentiment_counts.keys()
sizes = sentiment_counts.values()

# 颜色设置（草莓蛋糕风格）
colors = ["#FFB6C1", "#A52A2A", "#FFF5E1"]  # 粉色、深红色、奶油色
explode = (0.1, 0, 0)  # 突出显示“Sweet”部分

# 启用手绘风格
plt.xkcd()
# 自定义显示函数
def autopct_format(values):
    def my_format(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return f'{val}'  # 显示原始数字
    return my_format

# 绘制饼图
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct=autopct_format(sizes), startangle=90, shadow=True, wedgeprops={'edgecolor': 'black'})

# 设置英文标题
plt.title("Relationship Day Statistics", fontsize=16, fontweight="bold")

# 保存
plt.savefig("./mzz_data/relationship_day_statistics.png", dpi=300)

# 显示图表
plt.show()

In [ ]:
import json
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm 
import re 


def clearn_text(text):
    # 正则表达式匹配中文字符和标点符号，同时保留换行符
    pattern = re.compile(r'[^\u4e00-\u9fff\u3000-\u303f\uff00-\uffef\n]')

    # 替换掉非中文字符、标点符号和换行符
    filtered_text = re.sub(pattern, '', text)

    # 去除多余的空行（保留单行换行）
    filtered_text = re.sub(r'\n+', '\n', filtered_text).strip()
    return filtered_text
# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX",  # 替换为你的 API Key
    base_url="https://api.deepseek.com",
)


# 将 StrTime 转换为 datetime 类型
df['StrTime'] = pd.to_datetime(df['StrTime'])

template = """我正在制作我与女朋友的恋爱报告，我将给你我们某一天的对话内容，这一天的内容若含有非常甜蜜非常感人的句子，就将其输出为列表，若这一天的内容很平淡，就输出空列表。
以下是对话内容：
```conversation
{conversation}
```
最终请输出为json格式，格式示例为：
{{
    "老公": ["甜蜜的句子1", "甜蜜的句子2", ...],
    "老婆": ["甜蜜的句子1", "甜蜜的句子2", ...]
}}
"""

results = []
for date, df_1 in tqdm(df.groupby(["Date"]), total=df.groupby(["Date"]).ngroups):
    try:
        conversation = ""
        for _, line in df_1.iterrows():
            if line["Remark"] == "小老正":
                conversation += "老公：" + clearn_text(line["StrContent"]) + "\n"
            else:
                conversation += "老婆：" + clearn_text(line["StrContent"]) + "\n"
        prompt = template.format(conversation=conversation)
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        results.append(response.choices[0].message.content)
    except:
        continue    
    

In [ ]:
import json
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

def clearn_text(text):
    # 正则表达式匹配中文字符和标点符号，同时保留换行符
    pattern = re.compile(r'[^\u4e00-\u9fff\u3000-\u303f\uff00-\uffef\n]')

    # 替换掉非中文字符、标点符号和换行符
    filtered_text = re.sub(pattern, '', text)

    # 去除多余的空行（保留单行换行）
    filtered_text = re.sub(r'\n+', '\n', filtered_text).strip()
    return filtered_text

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX",  # 替换为你的 API Key
    base_url="https://api.deepseek.com",
)

# 假设 df 是你的 DataFrame
df['StrTime'] = pd.to_datetime(df['StrTime'])

template = """我正在制作我与女朋友的恋爱报告，我将给你我们某一天的对话内容。
这一天的内容若含有非常非常甜蜜感人的句子，就将其输出为列表，若这一天的内容很平淡，就输出空列表。（注意，由于我有上千条对话内容，因此请你严格筛选，只输出恋爱中非常非常甜蜜感人的句子）
以下是对话内容：
```conversation
{conversation}
```
最终请输出为json格式，格式示例为：
{{
    "老公": ["甜蜜的句子1", "甜蜜的句子2", ...],
    "老婆": ["甜蜜的句子1", "甜蜜的句子2", ...]
}}
"""

def process_conversation(date, df_1):
    try:
        conversation = ""
        for _, line in df_1.iterrows():
            if line["Remark"] == remark_1:
                conversation += "老公：" + clearn_text(line["StrContent"]) + "\n"
            else:
                conversation += "老婆：" + clearn_text(line["StrContent"]) + "\n"
        prompt = template.format(conversation=conversation)
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing date {date}: {e}")
        return None

results = []
with ThreadPoolExecutor(max_workers=10) as executor:  # 你可以根据需要调整 max_workers
    futures = {executor.submit(process_conversation, date, df_1): date for date, df_1 in df.groupby("Date")}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        date = futures[future]
        try:
            result = future.result()
            if result:
                results.append(result)
        except Exception as e:
            print(f"Error retrieving result for date {date}: {e}")

# 保存结果到文件或进行其他处理
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [32]:
mzz_sentences = []
for i in results:
    try:
        mzz_sentences += json.loads(i)["老公"]
    except:
        continue

syn_sentences = []
for i in results:
    try:
        syn_sentences += json.loads(i)["老婆"]
    except:
        continue

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX",  # 替换为你的 API Key
    base_url="https://api.deepseek.com",
)

template = """我正在制作我与女朋友的恋爱报告，你需要帮我判断给定的文本是否是真正甜蜜感人的话。

真正甜蜜的话示例：
- 我真的！想你了！
- 你是我向上的动力
- 你是我心中的小太阳
- 我真的好喜欢你呀

普通对话内容示例：
- 晚安
- 每个孩子我都特别特别喜欢
- 加油


以下是句子：
```sentence
{sentence}
```
最终请输出为json格式，格式示例为：
{{
    "判断": "是|否"
}}
"""

def process_sentence(sentence):
    try:
        prompt = template.format(sentence=sentence)
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        return sentence if result.get("判断") == "是" else None
    except Exception as e:
        print(f"Error processing sentence: {sentence}, error: {e}")
        return None

syn_sentences_processed = []
with ThreadPoolExecutor(max_workers=10) as executor:  # 调整 max_workers 控制并发数
    futures = {executor.submit(process_sentence, sentence): sentence for sentence in syn_sentences}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        sentence = futures[future]
        try:
            result = future.result()
            if result:
                syn_sentences_processed.append(result)
        except Exception as e:
            print(f"Error retrieving result for sentence {sentence}: {e}")

print("处理后的句子：", syn_sentences_processed)

In [49]:
with open("mzz_data/syn_sentences.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(syn_sentences_processed))

## 制作聊天记录卷轴

In [51]:
with open("mzz_data/mzz_sentences.txt", encoding="utf-8") as f:
    mzz_sentences = f.readlines()

with open("mzz_data/syn_sentences.txt", encoding="utf-8") as f:
    syn_sentences = f.readlines()

all_sentences = mzz_sentences + syn_sentences
print(f"共{all_sentences}条句子")

## 调整背景图的透明度

In [76]:
from PIL import Image

def adjust_image_transparency(image_path, output_path, transparency=0.5):
    """
    调整图片的透明度
    :param image_path: 输入图片路径
    :param output_path: 输出图片路径
    :param transparency: 透明度（0-1，0 为完全透明，1 为不透明）
    """
    image = Image.open(image_path).convert("RGBA")  # 转换为 RGBA 模式
    data = image.getdata()

    # 调整透明度
    new_data = []
    for item in data:
        # 修改透明度（alpha 通道）
        new_data.append((item[0], item[1], item[2], int(item[3] * transparency)))

    image.putdata(new_data)
    image.save(output_path, "PNG")  # 保存为 PNG 格式（支持透明度）

# 示例：将背景图片透明度调整为 50%
adjust_image_transparency("./mzz_data/background.png", "./mzz_data/background_transparent.png", transparency=0.2)

In [ ]:
import random
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT


# 1. 打乱列表顺序
random.shuffle(all_sentences)

# 2. 遍历列表元素，若句子长度大于15，将其均匀拆分成多个部分
def split_long_sentences(sentences, max_length=30):
    new_sentences = []
    for sentence in sentences:
        sentence = sentence.replace("\n", "")  # 去除原有换行符
        if len(sentence) <= max_length:
            new_sentences.append(sentence)
        else:
            # 计算需要拆分成几部分
            num_parts = (len(sentence) + max_length - 1) // max_length  # 向上取整
            part_length = (len(sentence) + num_parts - 1) // num_parts  # 均匀拆分
            # 均匀拆分句子
            split_parts = [sentence[i:i+part_length] for i in range(0, len(sentence), part_length)]
            new_sentences.extend(split_parts)  # 将拆分后的部分插入列表
    return new_sentences

formatted_sentences = split_long_sentences(all_sentences)

# 3. 写入 Word
def export_to_word(sentences, filename="./mzz_data/卷轴内容.docx"):
    doc = Document()
    
    # 设置字体（使用支持中文的字体，如宋体）
    style = doc.styles['Normal']
    font = style.font
    font.name = '宋体'
    font.size = Pt(12)
    font._element.rPr.rFonts.set(qn('w:eastAsia'), '宋体')  # 设置中文字体
    
    # 添加内容
    for sentence in sentences:
        paragraph = doc.add_paragraph()
        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # 居中对齐
        paragraph.add_run(sentence).bold = True  # 加粗
    
    # 保存文档
    doc.save(filename)
    print(f"文档已保存为 {filename}")

# 导出
export_to_word(formatted_sentences)

## 关键词提取及词云图绘制

In [ ]:
import json
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

def clearn_text(text):
    # 正则表达式匹配中文字符和标点符号，同时保留换行符
    pattern = re.compile(r'[^\u4e00-\u9fff\u3000-\u303f\uff00-\uffef\n]')

    # 替换掉非中文字符、标点符号和换行符
    filtered_text = re.sub(pattern, '', text)

    # 去除多余的空行（保留单行换行）
    filtered_text = re.sub(r'\n+', '\n', filtered_text).strip()
    return filtered_text

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="XXX",  # 替换为你的 API Key
    base_url="https://api.deepseek.com",
)

# 假设 df 是你的 DataFrame
df['StrTime'] = pd.to_datetime(df['StrTime'])

template = """我正在制作我与女朋友的恋爱报告，我想提取我们恋爱中经常提及的关键词。我将给你我们某一天的对话内容。
这一天的内容若含有非常非常甜蜜感人的关键词，就将其输出为列表，若这一天的内容很平淡，没什么可以提取的关键词，就输出空列表。（注意，由于我有上千条对话内容，因此请你严格筛选，只输出恋爱中非常非常甜蜜感人的关键词）
以下是对话内容：
```conversation
{conversation}
```
最终请输出为json格式，格式示例为：
{{
    "关键词": ["关键词1", "关键词2", ...],
}}
"""

def process_conversation(date, df_1):
    try:
        conversation = ""
        for _, line in df_1.iterrows():
            if line["Remark"] == remark_1:
                conversation += "老公：" + clearn_text(line["StrContent"]) + "\n"
            else:
                conversation += "老婆：" + clearn_text(line["StrContent"]) + "\n"
        prompt = template.format(conversation=conversation)
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing date {date}: {e}")
        return None

results = []
with ThreadPoolExecutor(max_workers=10) as executor:  # 你可以根据需要调整 max_workers
    futures = {executor.submit(process_conversation, date, df_1): date for date, df_1 in df.groupby("Date")}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        date = futures[future]
        try:
            result = future.result()
            if result:
                results.append(result)
        except Exception as e:
            print(f"Error retrieving result for date {date}: {e}")

# 保存结果到文件或进行其他处理
with open("./mzz_data/关键词.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [105]:
word_cloud_list = []
for i in results:
    try:
        word_cloud_list.extend(json.loads(i)["关键词"])
    except:
        pass
word_cloud_list = pd.Series(word_cloud_list).map(lambda x: x.strip()).tolist()

# 直接从word_cloud_list中删掉：你撤回了一条消息
word_cloud_list = [i for i in word_cloud_list if i != "你撤回了一条消息"]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from collections import Counter


# 统计词频
word_freq = Counter(word_cloud_list)

# 设置词云图样式
wordcloud = WordCloud(
    font_path='simhei.ttf',  # 使用支持中文的字体（如黑体）
    width=800,              # 图片宽度
    height=400,             # 图片高度
    background_color='white',  # 背景颜色
    # colormap='Pastel1',     # 使用淡粉色风格的颜色主题
    # colormap="inferno",
    prefer_horizontal=0.9,  # 水平排列的单词比例
    max_words=100,          # 最多显示的单词数量
    stopwords=stopwords,    # 设置停用词
    contour_width=1,        # 轮廓宽度
    contour_color='pink',   # 轮廓颜色
)

# 生成词云图
wordcloud.generate_from_frequencies(word_freq)

# 显示词云图
plt.figure(figsize=(10, 5))  # 设置画布大小
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # 隐藏坐标轴
plt.show()

# 保存词云图
wordcloud.to_file("./mzz_data/wordcloud_strawberry.png")
print("词云图已保存为 wordcloud_strawberry.png")